# 🔍 Lezione 26 — Anomaly Detection: Trovare l'Ago nel Pagliaio

## 📚 Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Capire** cosa sono le anomalie e perché rilevarle è importante
2. **Applicare** Isolation Forest per anomaly detection
3. **Usare** Local Outlier Factor (LOF) per rilevare outliers locali
4. **Confrontare** i diversi algoritmi e scegliere il più appropriato
5. **Valutare** le performance di un sistema di anomaly detection

---

## 🎯 Perché Questa Lezione è Importante

Le anomalie sono ovunque nel mondo reale:
- **Frodi bancarie:** transazioni sospette tra milioni di legittime
- **Cybersecurity:** attacchi tra traffico di rete normale
- **Manutenzione predittiva:** guasti imminenti in macchinari
- **Healthcare:** pazienti con condizioni rare
- **Quality control:** prodotti difettosi in produzione

**Il Problema:** Le anomalie sono RARE (1-5% dei dati)
- Non puoi usare classificazione supervisionata (poche label)
- Devi trovare pattern "strani" senza sapere cosa cercare

**La Soluzione:** Algoritmi unsupervised di Anomaly Detection!

---

## 📋 Struttura della Lezione

| Sezione | Contenuto |
|---------|-----------|
| **1. Teoria** | Tipi di anomalie, approcci, metriche |
| **2. Schema Mentale** | Workflow di anomaly detection |
| **3. Demo Pratiche** | 5 esempi hands-on |
| **4. Esercizi** | 3 esercizi con soluzioni |
| **5. Conclusione** | Best practices e prossimi passi |
| **6. Bignami** | Scheda di riferimento rapido |

---

# 📖 1. Teoria — Anomaly Detection Fondamenti

## 1.1 Cos'è un'Anomalia?

Un'**anomalia** (o outlier) è un'osservazione che si discosta significativamente dal comportamento normale.

```
📊 TIPI DI ANOMALIE:

1. POINT ANOMALIES (Puntuali)
   → Un singolo punto è anomalo
   → Es: Una transazione da 1M€ quando la media è 100€
   
2. CONTEXTUAL ANOMALIES (Contestuali)  
   → Anomalo solo in un certo contesto
   → Es: Temperatura 30°C è normale d'estate, anomala d'inverno
   
3. COLLECTIVE ANOMALIES (Collettive)
   → Un gruppo di punti insieme è anomalo
   → Es: Sequenza di 100 login falliti consecutivi
```

## 1.2 Approcci all'Anomaly Detection

```
📊 METODI PRINCIPALI:

┌─────────────────────────────────────────────────────────┐
│                ANOMALY DETECTION                         │
└─────────────────────────────────────────────────────────┘
         │
         ├── DENSITY-BASED
         │   └── LOF (Local Outlier Factor)
         │       → Confronta densità locale vs vicini
         │
         ├── DISTANCE-BASED  
         │   └── K-Nearest Neighbors
         │       → Punti lontani dai vicini = anomali
         │
         ├── ISOLATION-BASED
         │   └── Isolation Forest
         │       → Anomalie si isolano facilmente
         │
         └── RECONSTRUCTION-BASED
             └── Autoencoder
                 → Anomalie hanno alto errore di ricostruzione
```

---

## 1.3 Isolation Forest — L'Algoritmo Star

**Intuizione:** Le anomalie sono FACILI da isolare!

```
🌲 COME FUNZIONA ISOLATION FOREST:

1. Costruisce alberi decisionali CASUALI
   → Seleziona feature casuale
   → Seleziona split casuale tra min e max
   
2. Le ANOMALIE vengono isolate in POCHI SPLIT
   → Sono "diverse" → basta poco per separarle
   
3. I punti NORMALI richiedono MOLTI SPLIT
   → Sono "simili" → difficili da separare
   
4. ANOMALY SCORE = profondità media negli alberi
   → Bassa profondità = anomalia
   → Alta profondità = normale

📊 ESEMPIO VISIVO:

  Punti normali (cluster)     Anomalia
        ●●●●                     ★
        ●●●●
        ●●●●
        
  Split necessari: 10+        Split necessari: 2
```

### Parametri Chiave

| Parametro | Default | Descrizione |
|-----------|---------|-------------|
| `n_estimators` | 100 | Numero di alberi |
| `contamination` | 'auto' | % attesa di anomalie |
| `max_samples` | 'auto' | Campioni per albero |

---

## 1.4 Local Outlier Factor (LOF) — Densità Locale

**Intuizione:** Un punto è anomalo se la sua densità locale è MOLTO diversa da quella dei vicini.

```
📊 COME FUNZIONA LOF:

1. Per ogni punto, trova i K vicini più prossimi

2. Calcola la DENSITÀ LOCALE:
   → Quanti punti ci sono nel suo intorno?
   
3. Confronta con la densità dei VICINI:
   → Se io ho densità bassa ma i miei vicini alta → ANOMALIA
   
4. LOF SCORE:
   → LOF ≈ 1: normale (simile ai vicini)
   → LOF >> 1: anomalia (molto meno denso dei vicini)
   → LOF < 1: più denso dei vicini (super-normale)

📊 ESEMPIO:

   ●●●●●●        ★           ●●●●
   ●●●●●●                    ●●●●
   
   Cluster A    Anomalia    Cluster B
   (alta densità)  (LOF alto)  (alta densità)
```

### LOF vs Isolation Forest

| Aspetto | Isolation Forest | LOF |
|---------|-----------------|-----|
| **Velocità** | Molto veloce | Più lento (calcola distanze) |
| **Scalabilità** | Ottima (O(n log n)) | Media (O(n²)) |
| **Anomalie locali** | Meno sensibile | Eccellente |
| **Interpretabilità** | Media | Buona (score = densità) |

---

## 1.5 Metriche di Valutazione

Valutare anomaly detection è difficile perché spesso non abbiamo label!

```
📊 METRICHE CON LABEL (se disponibili):

┌──────────────────────────────────────────────────────────┐
│                    CONFUSION MATRIX                       │
│                                                          │
│                    Predetto                              │
│                 Normale  Anomalia                        │
│          ┌─────────┬──────────┐                          │
│ Reale    │   TN    │    FP    │  ← False Positive       │
│ Normale  │         │  (Allarme│     (Allerta inutile)   │
│          │         │   falso) │                          │
│          ├─────────┼──────────┤                          │
│ Reale    │   FN    │    TP    │  ← True Positive        │
│ Anomalia │ (Perso!)│  (Trovata│     (Anomalia rilevata) │
│          └─────────┴──────────┘                          │
└──────────────────────────────────────────────────────────┘

PRECISION = TP / (TP + FP)  → Quante anomalie predette sono vere?
RECALL    = TP / (TP + FN)  → Quante anomalie vere sono state trovate?
F1-SCORE  = 2 × (P × R) / (P + R)
```

### ⚠️ Problema: Classi Sbilanciate

Con 1% di anomalie:
- Predire sempre "normale" dà 99% accuracy ma è INUTILE
- Usa **Precision-Recall** invece di Accuracy
- Usa **F1-Score** o **AUC-PR**

---

# 🧠 2. Schema Mentale — Workflow Anomaly Detection

## 📊 Diagramma di Flusso

```
┌──────────────────────────────────────────────────────────────────────┐
│                    WORKFLOW ANOMALY DETECTION                         │
└──────────────────────────────────────────────────────────────────────┘

    ┌─────────────┐
    │ DATI GREZZI │
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐     Scaling necessario?
    │   SCALING   │ ◄── LOF: SÌ (usa distanze)
    └──────┬──────┘     Isolation Forest: OPZIONALE
           │
           ▼
    ┌─────────────┐     Quale algoritmo?
    │  ALGORITMO  │ ◄── Isolation Forest: veloce, general purpose
    └──────┬──────┘     LOF: anomalie locali
           │            DBSCAN: clustering + outliers
           ▼
    ┌─────────────┐     Come settare?
    │ CONTAMINATION│ ◄── % attesa di anomalie
    └──────┬──────┘     Se ignoto: prova 0.01-0.10
           │
           ▼
    ┌─────────────┐     Come valutare?
    │ VALUTAZIONE │ ◄── Con label: Precision, Recall, F1
    └──────┬──────┘     Senza label: Visualizzazione
           │
           ▼
    ┌─────────────┐
    │   OUTPUT    │ → Anomalie identificate + Scores
    └─────────────┘
```

## ✅ Checklist Anomaly Detection

- [ ] Definito cosa significa "anomalia" nel contesto?
- [ ] Stimata la % di anomalie attese?
- [ ] Scaling applicato (se necessario)?
- [ ] Algoritmo appropriato scelto?
- [ ] Soglia di decisione definita?
- [ ] Validazione effettuata?

---

# 🔬 3. Demo Pratiche

## 🎯 Demo 1 — Isolation Forest: Primo Esempio

Iniziamo con un dataset sintetico per capire come funziona Isolation Forest.

In [ ]:
# ============================================
# SETUP INIZIALE
# ============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, f1_score
from sklearn.datasets import make_blobs
import warnings
warnings.filterwarnings('ignore')

print("✅ Setup completato!")
print("="*70)

# ============================================
# DEMO 1 — Isolation Forest Base
# ============================================
print("\n" + "="*70)
print("DEMO 1 — Isolation Forest: Primo Esempio")
print("="*70)

# ============================================
# PASSO 1: Crea dataset con anomalie
# ============================================
np.random.seed(42)

# Dati normali: 2 cluster
X_normal, _ = make_blobs(n_samples=300, centers=2, cluster_std=0.5, random_state=42)

# Anomalie: punti sparsi random
n_anomalies = 20
X_anomalies = np.random.uniform(low=-6, high=6, size=(n_anomalies, 2))

# Combina
X = np.vstack([X_normal, X_anomalies])
y_true = np.array([0]*300 + [1]*n_anomalies)  # 0=normale, 1=anomalia

print(f"\n📊 Dataset:")
print(f"   - Punti normali: 300")
print(f"   - Anomalie: {n_anomalies}")
print(f"   - Contamination reale: {n_anomalies/(300+n_anomalies)*100:.1f}%")

# ============================================
# PASSO 2: Applica Isolation Forest
# ============================================
# contamination = % attesa di anomalie
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 5% anomalie attese
    random_state=42
)

# fit_predict ritorna: 1 = normale, -1 = anomalia
y_pred_iso = iso_forest.fit_predict(X)

# Converti a 0/1 per confronto con y_true
y_pred_binary = (y_pred_iso == -1).astype(int)  # -1 → 1 (anomalia)

print(f"\n📈 Isolation Forest Results:")
print(f"   - Anomalie rilevate: {(y_pred_iso == -1).sum()}")
print(f"   - Anomalie reali: {y_true.sum()}")

# ============================================
# PASSO 3: Valutazione
# ============================================
# Calcola metriche
tp = np.sum((y_true == 1) & (y_pred_binary == 1))
fp = np.sum((y_true == 0) & (y_pred_binary == 1))
fn = np.sum((y_true == 1) & (y_pred_binary == 0))

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"\n📊 Performance:")
print(f"   - Precision: {precision:.3f}")
print(f"   - Recall: {recall:.3f}")
print(f"   - F1-Score: {f1:.3f}")

# ============================================
# PASSO 4: Visualizzazione
# ============================================
# Ottieni anomaly scores
scores = iso_forest.decision_function(X)  # Più basso = più anomalo

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Ground Truth
colors_true = ['blue' if y == 0 else 'red' for y in y_true]
axes[0].scatter(X[:, 0], X[:, 1], c=colors_true, s=30, alpha=0.7)
axes[0].set_title('Ground Truth\n(rosso = anomalie reali)', fontsize=12)
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# Plot 2: Predizioni Isolation Forest
colors_pred = ['blue' if y == 1 else 'red' for y in y_pred_iso]
axes[1].scatter(X[:, 0], X[:, 1], c=colors_pred, s=30, alpha=0.7)
axes[1].set_title(f'Isolation Forest\nPrecision={precision:.2f}, Recall={recall:.2f}', fontsize=12)
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')

# Plot 3: Anomaly Scores
scatter = axes[2].scatter(X[:, 0], X[:, 1], c=scores, cmap='RdYlBu', s=30, alpha=0.7)
axes[2].set_title('Anomaly Scores\n(rosso = più anomalo)', fontsize=12)
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')
plt.colorbar(scatter, ax=axes[2], label='Score')

plt.tight_layout()
plt.show()

print("""
📌 OSSERVAZIONI:
   - Isolation Forest rileva bene le anomalie sparse
   - Lo score indica quanto è "anomalo" ogni punto
   - Alcuni punti al bordo dei cluster sono erroneamente classificati
""")

---

## 🎯 Demo 2 — Local Outlier Factor (LOF)

LOF è eccellente per rilevare anomalie **locali** — punti anomali rispetto ai loro vicini.

In [ ]:
# ============================================
# DEMO 2 — Local Outlier Factor (LOF)
# ============================================

print("="*70)
print("DEMO 2 — Local Outlier Factor (LOF)")
print("="*70)

# ============================================
# PASSO 1: Dataset con anomalie LOCALI
# ============================================
np.random.seed(42)

# Cluster denso
cluster_dense = np.random.randn(200, 2) * 0.3

# Cluster sparso
cluster_sparse = np.random.randn(100, 2) * 2 + [5, 5]

# Anomalie: punti tra i due cluster (localmente anomali!)
# Questi sono normali globalmente ma anomali localmente
local_anomalies = np.array([
    [2.5, 2.5],
    [2.0, 3.0],
    [3.0, 2.0],
    [-2, -2],
    [-2.5, -1.5]
])

# Anomalie globali
global_anomalies = np.array([
    [10, 10],
    [-8, 8],
    [8, -8]
])

X_lof = np.vstack([cluster_dense, cluster_sparse, local_anomalies, global_anomalies])
y_true_lof = np.array([0]*200 + [0]*100 + [1]*5 + [1]*3)  # 8 anomalie totali

print(f"\n📊 Dataset:")
print(f"   - Cluster denso: 200 punti")
print(f"   - Cluster sparso: 100 punti")
print(f"   - Anomalie locali: 5")
print(f"   - Anomalie globali: 3")

# ============================================
# PASSO 2: Applica LOF
# ============================================
# n_neighbors: quanti vicini considerare
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.03,
    novelty=False  # False per fit_predict
)

y_pred_lof = lof.fit_predict(X_lof)
y_pred_lof_binary = (y_pred_lof == -1).astype(int)

# Ottieni LOF scores
lof_scores = -lof.negative_outlier_factor_  # Più alto = più anomalo

print(f"\n📈 LOF Results:")
print(f"   - Anomalie rilevate: {(y_pred_lof == -1).sum()}")
print(f"   - Anomalie reali: {y_true_lof.sum()}")

# ============================================
# PASSO 3: Valutazione
# ============================================
tp_lof = np.sum((y_true_lof == 1) & (y_pred_lof_binary == 1))
fp_lof = np.sum((y_true_lof == 0) & (y_pred_lof_binary == 1))
fn_lof = np.sum((y_true_lof == 1) & (y_pred_lof_binary == 0))

precision_lof = tp_lof / (tp_lof + fp_lof) if (tp_lof + fp_lof) > 0 else 0
recall_lof = tp_lof / (tp_lof + fn_lof) if (tp_lof + fn_lof) > 0 else 0
f1_lof = 2 * precision_lof * recall_lof / (precision_lof + recall_lof) if (precision_lof + recall_lof) > 0 else 0

print(f"\n📊 Performance LOF:")
print(f"   - Precision: {precision_lof:.3f}")
print(f"   - Recall: {recall_lof:.3f}")
print(f"   - F1-Score: {f1_lof:.3f}")

# ============================================
# PASSO 4: Confronto con Isolation Forest
# ============================================
iso_forest_compare = IsolationForest(contamination=0.03, random_state=42)
y_pred_iso_compare = iso_forest_compare.fit_predict(X_lof)
y_pred_iso_binary = (y_pred_iso_compare == -1).astype(int)

tp_iso = np.sum((y_true_lof == 1) & (y_pred_iso_binary == 1))
fp_iso = np.sum((y_true_lof == 0) & (y_pred_iso_binary == 1))
fn_iso = np.sum((y_true_lof == 1) & (y_pred_iso_binary == 0))

precision_iso = tp_iso / (tp_iso + fp_iso) if (tp_iso + fp_iso) > 0 else 0
recall_iso = tp_iso / (tp_iso + fn_iso) if (tp_iso + fn_iso) > 0 else 0
f1_iso = 2 * precision_iso * recall_iso / (precision_iso + recall_iso) if (precision_iso + recall_iso) > 0 else 0

print(f"\n📊 Performance Isolation Forest:")
print(f"   - Precision: {precision_iso:.3f}")
print(f"   - Recall: {recall_iso:.3f}")
print(f"   - F1-Score: {f1_iso:.3f}")

# ============================================
# PASSO 5: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Ground truth
colors_true_lof = ['blue' if y == 0 else 'red' for y in y_true_lof]
axes[0].scatter(X_lof[:, 0], X_lof[:, 1], c=colors_true_lof, s=30, alpha=0.7)
axes[0].set_title('Ground Truth', fontsize=12)
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# LOF
colors_lof = ['blue' if y == 1 else 'red' for y in y_pred_lof]
axes[1].scatter(X_lof[:, 0], X_lof[:, 1], c=colors_lof, s=30, alpha=0.7)
axes[1].set_title(f'LOF\nF1={f1_lof:.2f}', fontsize=12)
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')

# Isolation Forest
colors_iso_c = ['blue' if y == 1 else 'red' for y in y_pred_iso_compare]
axes[2].scatter(X_lof[:, 0], X_lof[:, 1], c=colors_iso_c, s=30, alpha=0.7)
axes[2].set_title(f'Isolation Forest\nF1={f1_iso:.2f}', fontsize=12)
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

print("""
📌 OSSERVAZIONI:
   - LOF rileva meglio le anomalie LOCALI (tra i cluster)
   - Isolation Forest può perdere anomalie locali
   - LOF considera la densità dei vicini, non solo la posizione globale
""")

---

## 🎯 Demo 3 — Tuning del Parametro Contamination

Il parametro `contamination` è cruciale: indica la % di anomalie attese.

In [ ]:
# ============================================
# DEMO 3 — Tuning Contamination
# ============================================

print("="*70)
print("DEMO 3 — Impatto del Parametro Contamination")
print("="*70)

# Usiamo il dataset della Demo 1
contamination_values = [0.01, 0.05, 0.10, 0.15, 0.20]

results_contamination = []

print("\n📊 Test con diversi valori di contamination:")
print("-" * 60)

for cont in contamination_values:
    iso = IsolationForest(n_estimators=100, contamination=cont, random_state=42)
    y_pred = iso.fit_predict(X)
    y_pred_bin = (y_pred == -1).astype(int)
    
    n_detected = (y_pred == -1).sum()
    
    tp = np.sum((y_true == 1) & (y_pred_bin == 1))
    fp = np.sum((y_true == 0) & (y_pred_bin == 1))
    fn = np.sum((y_true == 1) & (y_pred_bin == 0))
    
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    
    results_contamination.append({
        'contamination': cont,
        'detected': n_detected,
        'precision': prec,
        'recall': rec,
        'f1': f1
    })
    
    print(f"   cont={cont:.2f}: detected={n_detected:3d}, P={prec:.2f}, R={rec:.2f}, F1={f1:.2f}")

# ============================================
# Visualizzazione
# ============================================
df_cont = pd.DataFrame(results_contamination)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Anomalie rilevate
axes[0].bar(range(len(contamination_values)), df_cont['detected'], color='steelblue', alpha=0.7)
axes[0].axhline(y=20, color='red', linestyle='--', label=f'Anomalie reali: 20')
axes[0].set_xticks(range(len(contamination_values)))
axes[0].set_xticklabels([f'{c:.0%}' for c in contamination_values])
axes[0].set_xlabel('Contamination')
axes[0].set_ylabel('Anomalie Rilevate')
axes[0].set_title('Numero Anomalie Rilevate vs Contamination')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Metriche
axes[1].plot(contamination_values, df_cont['precision'], 'bo-', label='Precision', linewidth=2)
axes[1].plot(contamination_values, df_cont['recall'], 'go-', label='Recall', linewidth=2)
axes[1].plot(contamination_values, df_cont['f1'], 'ro-', label='F1-Score', linewidth=2)
axes[1].set_xlabel('Contamination')
axes[1].set_ylabel('Score')
axes[1].set_title('Precision, Recall, F1 vs Contamination')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Trova il migliore
best_idx = df_cont['f1'].idxmax()
best_cont = df_cont.loc[best_idx, 'contamination']
best_f1 = df_cont.loc[best_idx, 'f1']

print(f"""
📌 OSSERVAZIONI:
   - Contamination basso → Alta Precision, Bassa Recall (perse anomalie)
   - Contamination alto → Alta Recall, Bassa Precision (troppi falsi positivi)
   - Miglior F1 con contamination={best_cont:.2f} (F1={best_f1:.2f})
   
💡 REGOLA: Se non sai la % reale, inizia con 0.01-0.05 e adatta
""")

---

## 🎯 Demo 4 — Anomaly Detection su Dataset Reale

Applichiamo anomaly detection al dataset Credit Card Fraud (simulato).

In [ ]:
# ============================================
# DEMO 4 — Dataset Realistico: Credit Card Fraud
# ============================================

print("="*70)
print("DEMO 4 — Simulazione Credit Card Fraud Detection")
print("="*70)

# ============================================
# PASSO 1: Crea dataset simile a frodi bancarie
# ============================================
np.random.seed(42)

n_normal = 5000
n_fraud = 100  # 2% frodi

# Transazioni normali: importi bassi-medi, orari diurni
normal_amount = np.abs(np.random.exponential(50, n_normal))  # Media 50€
normal_hour = np.random.normal(14, 4, n_normal) % 24  # Picco ore 14
normal_freq = np.random.poisson(5, n_normal)  # 5 transazioni/settimana
normal_distance = np.abs(np.random.normal(20, 10, n_normal))  # 20km da casa

# Frodi: importi alti, orari notturni, distanza anomala
fraud_amount = np.random.uniform(500, 5000, n_fraud)  # 500-5000€
fraud_hour = np.random.uniform(0, 6, n_fraud)  # Notte
fraud_freq = np.random.uniform(15, 30, n_fraud)  # Molte transazioni
fraud_distance = np.random.uniform(500, 2000, n_fraud)  # Lontano da casa

# Combina le feature
X_fraud = np.column_stack([
    np.concatenate([normal_amount, fraud_amount]),
    np.concatenate([normal_hour, fraud_hour]),
    np.concatenate([normal_freq, fraud_freq]),
    np.concatenate([normal_distance, fraud_distance])
])

y_fraud = np.array([0]*n_normal + [1]*n_fraud)

feature_names_fraud = ['Importo (€)', 'Ora', 'Frequenza', 'Distanza (km)']

print(f"\n📊 Dataset Credit Card:")
print(f"   - Transazioni normali: {n_normal}")
print(f"   - Frodi: {n_fraud}")
print(f"   - Fraud rate: {n_fraud/(n_normal+n_fraud)*100:.1f}%")
print(f"   - Feature: {feature_names_fraud}")

# ============================================
# PASSO 2: Preprocessing
# ============================================
scaler = StandardScaler()
X_fraud_scaled = scaler.fit_transform(X_fraud)

# ============================================
# PASSO 3: Applica Isolation Forest e LOF
# ============================================
# Isolation Forest
iso_fraud = IsolationForest(contamination=0.02, random_state=42)
y_pred_iso_fraud = iso_fraud.fit_predict(X_fraud_scaled)
y_pred_iso_bin = (y_pred_iso_fraud == -1).astype(int)

# LOF
lof_fraud = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
y_pred_lof_fraud = lof_fraud.fit_predict(X_fraud_scaled)
y_pred_lof_bin = (y_pred_lof_fraud == -1).astype(int)

# ============================================
# PASSO 4: Valutazione
# ============================================
def calc_metrics(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': prec, 'Recall': rec, 'F1': f1}

metrics_iso = calc_metrics(y_fraud, y_pred_iso_bin)
metrics_lof = calc_metrics(y_fraud, y_pred_lof_bin)

print("\n" + "="*50)
print("CONFRONTO RISULTATI")
print("="*50)

comparison_fraud = pd.DataFrame({
    'Metrica': ['TP (Frodi trovate)', 'FP (Falsi allarmi)', 'FN (Frodi perse)', 
                'Precision', 'Recall', 'F1-Score'],
    'Isolation Forest': [metrics_iso['TP'], metrics_iso['FP'], metrics_iso['FN'],
                         f"{metrics_iso['Precision']:.3f}", 
                         f"{metrics_iso['Recall']:.3f}",
                         f"{metrics_iso['F1']:.3f}"],
    'LOF': [metrics_lof['TP'], metrics_lof['FP'], metrics_lof['FN'],
            f"{metrics_lof['Precision']:.3f}",
            f"{metrics_lof['Recall']:.3f}",
            f"{metrics_lof['F1']:.3f}"]
})
print(comparison_fraud.to_string(index=False))

# ============================================
# PASSO 5: Visualizzazione
# ============================================
from sklearn.decomposition import PCA

# Riduci a 2D per visualizzazione
pca_fraud = PCA(n_components=2)
X_fraud_2d = pca_fraud.fit_transform(X_fraud_scaled)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Ground truth
colors_gt = ['blue' if y == 0 else 'red' for y in y_fraud]
axes[0].scatter(X_fraud_2d[:, 0], X_fraud_2d[:, 1], c=colors_gt, s=5, alpha=0.5)
axes[0].set_title(f'Ground Truth\n({n_fraud} frodi reali)', fontsize=12)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

# Isolation Forest
colors_iso_f = ['blue' if y == 1 else 'red' for y in y_pred_iso_fraud]
axes[1].scatter(X_fraud_2d[:, 0], X_fraud_2d[:, 1], c=colors_iso_f, s=5, alpha=0.5)
axes[1].set_title(f'Isolation Forest\nF1={metrics_iso["F1"]:.2f}', fontsize=12)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

# LOF
colors_lof_f = ['blue' if y == 1 else 'red' for y in y_pred_lof_fraud]
axes[2].scatter(X_fraud_2d[:, 0], X_fraud_2d[:, 1], c=colors_lof_f, s=5, alpha=0.5)
axes[2].set_title(f'LOF\nF1={metrics_lof["F1"]:.2f}', fontsize=12)
axes[2].set_xlabel('PC1')
axes[2].set_ylabel('PC2')

plt.tight_layout()
plt.show()

print(f"""
📌 INTERPRETAZIONE BUSINESS:
   - Su 100 frodi reali, Isolation Forest ne trova {metrics_iso['TP']}
   - {metrics_iso['FP']} falsi allarmi da investigare manualmente
   - {metrics_iso['FN']} frodi perse (potenziale danno economico!)
   
💡 Nel fraud detection, spesso RECALL è più importante di PRECISION
   (meglio investigare falsi allarmi che perdere frodi)
""")

---

## 🎯 Demo 5 — Ensemble: Combinare Più Algoritmi

Combinare più algoritmi spesso migliora la detection.

In [ ]:
# ============================================
# DEMO 5 — Ensemble di Anomaly Detectors
# ============================================

print("="*70)
print("DEMO 5 — Ensemble: Combinare Isolation Forest + LOF")
print("="*70)

# ============================================
# Strategia: Un punto è anomalia se ENTRAMBI gli algoritmi concordano
# Oppure: Voting a maggioranza
# ============================================

# Riusiamo il dataset fraud
# y_pred_iso_bin e y_pred_lof_bin già calcolati

# ============================================
# STRATEGIA 1: AND (entrambi devono dire anomalia)
# ============================================
y_pred_and = ((y_pred_iso_bin == 1) & (y_pred_lof_bin == 1)).astype(int)

metrics_and = calc_metrics(y_fraud, y_pred_and)

print("\n" + "="*50)
print("STRATEGIA AND (più conservativa)")
print("="*50)
print(f"   - Anomalie trovate: {y_pred_and.sum()}")
print(f"   - Precision: {metrics_and['Precision']:.3f}")
print(f"   - Recall: {metrics_and['Recall']:.3f}")
print(f"   - F1-Score: {metrics_and['F1']:.3f}")

# ============================================
# STRATEGIA 2: OR (almeno uno dice anomalia)
# ============================================
y_pred_or = ((y_pred_iso_bin == 1) | (y_pred_lof_bin == 1)).astype(int)

metrics_or = calc_metrics(y_fraud, y_pred_or)

print("\n" + "="*50)
print("STRATEGIA OR (più aggressiva)")
print("="*50)
print(f"   - Anomalie trovate: {y_pred_or.sum()}")
print(f"   - Precision: {metrics_or['Precision']:.3f}")
print(f"   - Recall: {metrics_or['Recall']:.3f}")
print(f"   - F1-Score: {metrics_or['F1']:.3f}")

# ============================================
# STRATEGIA 3: Score averaging
# ============================================
# Ottieni gli scores
iso_scores_fraud = -iso_fraud.decision_function(X_fraud_scaled)  # Più alto = più anomalo
lof_scores_fraud = -lof_fraud.negative_outlier_factor_

# Normalizza a [0, 1]
iso_scores_norm = (iso_scores_fraud - iso_scores_fraud.min()) / (iso_scores_fraud.max() - iso_scores_fraud.min())
lof_scores_norm = (lof_scores_fraud - lof_scores_fraud.min()) / (lof_scores_fraud.max() - lof_scores_fraud.min())

# Media degli scores
ensemble_scores = (iso_scores_norm + lof_scores_norm) / 2

# Threshold: top 2% come anomalie
threshold = np.percentile(ensemble_scores, 98)
y_pred_ensemble = (ensemble_scores >= threshold).astype(int)

metrics_ensemble = calc_metrics(y_fraud, y_pred_ensemble)

print("\n" + "="*50)
print("STRATEGIA SCORE AVERAGING")
print("="*50)
print(f"   - Anomalie trovate: {y_pred_ensemble.sum()}")
print(f"   - Precision: {metrics_ensemble['Precision']:.3f}")
print(f"   - Recall: {metrics_ensemble['Recall']:.3f}")
print(f"   - F1-Score: {metrics_ensemble['F1']:.3f}")

# ============================================
# Confronto finale
# ============================================
print("\n" + "="*70)
print("CONFRONTO TUTTE LE STRATEGIE")
print("="*70)

all_results = pd.DataFrame({
    'Strategia': ['Isolation Forest', 'LOF', 'AND', 'OR', 'Score Avg'],
    'Detected': [(y_pred_iso_bin==1).sum(), (y_pred_lof_bin==1).sum(), 
                 y_pred_and.sum(), y_pred_or.sum(), y_pred_ensemble.sum()],
    'Precision': [metrics_iso['Precision'], metrics_lof['Precision'],
                  metrics_and['Precision'], metrics_or['Precision'], metrics_ensemble['Precision']],
    'Recall': [metrics_iso['Recall'], metrics_lof['Recall'],
               metrics_and['Recall'], metrics_or['Recall'], metrics_ensemble['Recall']],
    'F1': [metrics_iso['F1'], metrics_lof['F1'],
           metrics_and['F1'], metrics_or['F1'], metrics_ensemble['F1']]
})
print(all_results.to_string(index=False))

# Visualizzazione
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(all_results))
width = 0.25

bars1 = ax.bar(x - width, all_results['Precision'], width, label='Precision', color='steelblue')
bars2 = ax.bar(x, all_results['Recall'], width, label='Recall', color='forestgreen')
bars3 = ax.bar(x + width, all_results['F1'], width, label='F1-Score', color='coral')

ax.set_xlabel('Strategia')
ax.set_ylabel('Score')
ax.set_title('Confronto Strategie di Ensemble')
ax.set_xticks(x)
ax.set_xticklabels(all_results['Strategia'], rotation=15)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

best_strategy = all_results.loc[all_results['F1'].idxmax(), 'Strategia']
print(f"""
📌 CONCLUSIONI:
   - AND: Alta Precision, Bassa Recall (conservativo)
   - OR: Alta Recall, Bassa Precision (aggressivo)  
   - Score Avg: Buon bilanciamento
   - Miglior F1: {best_strategy}
   
💡 La scelta dipende dal costo di FP vs FN nel tuo business!
""")

---

# 🏋️ 4. Esercizi Pratici

## 🏋️ Esercizio 26.1 — Anomaly Detection su Dati Sensori

**Obiettivo:** Simulare il monitoraggio di un macchinario industriale.

**Consegna:**
1. Crea un dataset di **10.000 letture** da sensori con 5 feature:
   - Temperatura, Pressione, Vibrazione, RPM, Consumo energia
2. Aggiungi **100 anomalie** (valori fuori range)
3. Applica **Isolation Forest** con contamination corretto
4. Calcola **Precision, Recall, F1**
5. Visualizza le anomalie rilevate vs ground truth

In [ ]:
# ============================================
# ESERCIZIO 26.1 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 26.1 — Monitoraggio Sensori Industriali")
print("="*70)

# ============================================
# PASSO 1: Crea dataset sensori
# ============================================
np.random.seed(42)

n_normal_sensors = 10000
n_anomaly_sensors = 100

# Letture normali (macchinario funzionante)
temp_normal = np.random.normal(70, 5, n_normal_sensors)  # 70°C ± 5
pressure_normal = np.random.normal(100, 10, n_normal_sensors)  # 100 bar ± 10
vibration_normal = np.random.normal(0.5, 0.1, n_normal_sensors)  # 0.5 mm/s ± 0.1
rpm_normal = np.random.normal(3000, 100, n_normal_sensors)  # 3000 rpm ± 100
energy_normal = np.random.normal(50, 5, n_normal_sensors)  # 50 kW ± 5

# Anomalie (guasti imminenti!)
temp_anomaly = np.random.uniform(100, 150, n_anomaly_sensors)  # Surriscaldamento
pressure_anomaly = np.random.uniform(150, 200, n_anomaly_sensors)  # Pressione alta
vibration_anomaly = np.random.uniform(2, 5, n_anomaly_sensors)  # Vibrazione anomala
rpm_anomaly = np.random.uniform(1000, 2000, n_anomaly_sensors)  # RPM basso
energy_anomaly = np.random.uniform(80, 120, n_anomaly_sensors)  # Consumo alto

# Combina
X_sensors = np.column_stack([
    np.concatenate([temp_normal, temp_anomaly]),
    np.concatenate([pressure_normal, pressure_anomaly]),
    np.concatenate([vibration_normal, vibration_anomaly]),
    np.concatenate([rpm_normal, rpm_anomaly]),
    np.concatenate([energy_normal, energy_anomaly])
])

y_sensors = np.array([0]*n_normal_sensors + [1]*n_anomaly_sensors)
sensor_names = ['Temperatura', 'Pressione', 'Vibrazione', 'RPM', 'Energia']

print(f"\n📊 Dataset Sensori:")
print(f"   - Letture normali: {n_normal_sensors}")
print(f"   - Anomalie (guasti): {n_anomaly_sensors}")
print(f"   - Contamination: {n_anomaly_sensors/(n_normal_sensors+n_anomaly_sensors)*100:.1f}%")

# ============================================
# PASSO 2: Preprocessing
# ============================================
scaler_sensors = StandardScaler()
X_sensors_scaled = scaler_sensors.fit_transform(X_sensors)

# ============================================
# PASSO 3: Isolation Forest
# ============================================
iso_sensors = IsolationForest(
    n_estimators=100,
    contamination=0.01,  # 1% anomalie attese
    random_state=42
)

y_pred_sensors = iso_sensors.fit_predict(X_sensors_scaled)
y_pred_sensors_bin = (y_pred_sensors == -1).astype(int)

# ============================================
# PASSO 4: Valutazione
# ============================================
metrics_sensors = calc_metrics(y_sensors, y_pred_sensors_bin)

print(f"\n📈 Risultati Isolation Forest:")
print(f"   - Anomalie rilevate: {y_pred_sensors_bin.sum()}")
print(f"   - Anomalie reali: {y_sensors.sum()}")
print(f"   - Precision: {metrics_sensors['Precision']:.3f}")
print(f"   - Recall: {metrics_sensors['Recall']:.3f}")
print(f"   - F1-Score: {metrics_sensors['F1']:.3f}")

# ============================================
# PASSO 5: Visualizzazione
# ============================================
# PCA per visualizzazione
from sklearn.decomposition import PCA
pca_sensors = PCA(n_components=2)
X_sensors_2d = pca_sensors.fit_transform(X_sensors_scaled)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Ground truth
colors_gt_s = ['blue' if y == 0 else 'red' for y in y_sensors]
axes[0].scatter(X_sensors_2d[:, 0], X_sensors_2d[:, 1], c=colors_gt_s, s=3, alpha=0.5)
axes[0].set_title('Ground Truth (rosso = guasti)', fontsize=12)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

# Predizioni
colors_pred_s = ['blue' if y == 1 else 'red' for y in y_pred_sensors]
axes[1].scatter(X_sensors_2d[:, 0], X_sensors_2d[:, 1], c=colors_pred_s, s=3, alpha=0.5)
axes[1].set_title(f'Isolation Forest\nF1={metrics_sensors["F1"]:.2f}', fontsize=12)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

# Distribuzione scores
scores_sensors = -iso_sensors.decision_function(X_sensors_scaled)
axes[2].hist(scores_sensors[y_sensors==0], bins=50, alpha=0.7, label='Normale', color='blue')
axes[2].hist(scores_sensors[y_sensors==1], bins=50, alpha=0.7, label='Anomalia', color='red')
axes[2].set_xlabel('Anomaly Score')
axes[2].set_ylabel('Frequenza')
axes[2].set_title('Distribuzione Anomaly Scores', fontsize=12)
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"""
📌 INTERPRETAZIONE:
   - Le anomalie hanno scores più alti (più "anomali")
   - L'algoritmo rileva {metrics_sensors['TP']} guasti su {n_anomaly_sensors}
   - In produzione, queste anomalie attiverebbero un allarme!
""")

---

## 🏋️ Esercizio 26.2 — Confronto LOF vs Isolation Forest

**Obiettivo:** Determinare quale algoritmo è migliore per diversi tipi di anomalie.

**Consegna:**
1. Crea un dataset con:
   - **Anomalie globali** (punti lontani da tutti)
   - **Anomalie locali** (punti in zone a bassa densità tra cluster)
2. Applica **Isolation Forest** e **LOF**
3. Confronta quale algoritmo rileva meglio quale tipo di anomalia
4. Visualizza i risultati

In [ ]:
# ============================================
# ESERCIZIO 26.2 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 26.2 — Confronto LOF vs Isolation Forest")
print("="*70)

# ============================================
# PASSO 1: Crea dataset con anomalie globali e locali
# ============================================
np.random.seed(42)

# Due cluster con densità diversa
cluster_dense = np.random.randn(500, 2) * 0.5  # Denso
cluster_sparse = np.random.randn(500, 2) * 1.5 + [6, 6]  # Più sparso

# Anomalie GLOBALI: lontane da tutto
global_anom = np.array([
    [15, 15], [-10, 10], [10, -10], [-10, -10], [0, 15]
])

# Anomalie LOCALI: tra i cluster (localmente anomale)
local_anom = np.array([
    [3, 3], [2.5, 3.5], [3.5, 2.5], [4, 4], [2, 4]
])

X_compare = np.vstack([cluster_dense, cluster_sparse, global_anom, local_anom])

# Labels: 0=normale, 1=global, 2=local
y_type = np.array([0]*500 + [0]*500 + [1]*5 + [2]*5)
y_anomaly = (y_type > 0).astype(int)  # 0=normale, 1=anomalia (qualsiasi tipo)

print(f"\n📊 Dataset:")
print(f"   - Punti normali: 1000")
print(f"   - Anomalie globali: 5")
print(f"   - Anomalie locali: 5")

# ============================================
# PASSO 2: Applica Isolation Forest e LOF
# ============================================
contamination_rate = 10 / 1010  # ~1%

# Isolation Forest
iso_compare = IsolationForest(contamination=contamination_rate, random_state=42)
y_pred_iso_c = iso_compare.fit_predict(X_compare)
y_pred_iso_c_bin = (y_pred_iso_c == -1).astype(int)

# LOF
lof_compare = LocalOutlierFactor(n_neighbors=20, contamination=contamination_rate)
y_pred_lof_c = lof_compare.fit_predict(X_compare)
y_pred_lof_c_bin = (y_pred_lof_c == -1).astype(int)

# ============================================
# PASSO 3: Analisi per tipo di anomalia
# ============================================
print("\n" + "="*50)
print("DETECTION PER TIPO DI ANOMALIA")
print("="*50)

# Anomalie globali rilevate
global_mask = (y_type == 1)
iso_global_detected = y_pred_iso_c_bin[global_mask].sum()
lof_global_detected = y_pred_lof_c_bin[global_mask].sum()

# Anomalie locali rilevate
local_mask = (y_type == 2)
iso_local_detected = y_pred_iso_c_bin[local_mask].sum()
lof_local_detected = y_pred_lof_c_bin[local_mask].sum()

print(f"\n📊 Anomalie GLOBALI (5 totali):")
print(f"   - Isolation Forest: {iso_global_detected}/5 rilevate")
print(f"   - LOF: {lof_global_detected}/5 rilevate")

print(f"\n📊 Anomalie LOCALI (5 totali):")
print(f"   - Isolation Forest: {iso_local_detected}/5 rilevate")
print(f"   - LOF: {lof_local_detected}/5 rilevate")

# Metriche globali
metrics_iso_c = calc_metrics(y_anomaly, y_pred_iso_c_bin)
metrics_lof_c = calc_metrics(y_anomaly, y_pred_lof_c_bin)

print(f"\n📈 Metriche complessive:")
print(f"   Isolation Forest: F1={metrics_iso_c['F1']:.3f}")
print(f"   LOF: F1={metrics_lof_c['F1']:.3f}")

# ============================================
# PASSO 4: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Ground truth con tipi
colors_type = {0: 'blue', 1: 'red', 2: 'orange'}
colors_gt_c = [colors_type[y] for y in y_type]
axes[0].scatter(X_compare[:, 0], X_compare[:, 1], c=colors_gt_c, s=20, alpha=0.7)
axes[0].scatter([], [], c='red', label='Globali', s=50)
axes[0].scatter([], [], c='orange', label='Locali', s=50)
axes[0].legend()
axes[0].set_title('Ground Truth\n(rosso=globali, arancione=locali)', fontsize=11)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')

# Isolation Forest
colors_iso_det = ['red' if y == 1 else 'blue' for y in y_pred_iso_c_bin]
axes[1].scatter(X_compare[:, 0], X_compare[:, 1], c=colors_iso_det, s=20, alpha=0.7)
axes[1].set_title(f'Isolation Forest\nGlobali: {iso_global_detected}/5, Locali: {iso_local_detected}/5', fontsize=11)
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')

# LOF
colors_lof_det = ['red' if y == 1 else 'blue' for y in y_pred_lof_c_bin]
axes[2].scatter(X_compare[:, 0], X_compare[:, 1], c=colors_lof_det, s=20, alpha=0.7)
axes[2].set_title(f'LOF\nGlobali: {lof_global_detected}/5, Locali: {lof_local_detected}/5', fontsize=11)
axes[2].set_xlabel('X')
axes[2].set_ylabel('Y')

plt.tight_layout()
plt.show()

print(f"""
📌 CONCLUSIONI:
   - Isolation Forest: eccellente per anomalie GLOBALI
   - LOF: migliore per anomalie LOCALI (considera densità vicini)
   
💡 REGOLA PRATICA:
   - Usa Isolation Forest come default (veloce, robusto)
   - Usa LOF se sospetti anomalie locali/contestuali
   - Combina entrambi per massima copertura
""")

---

## 🏋️ Esercizio 26.3 — Threshold Optimization

**Obiettivo:** Trovare la soglia ottimale per bilanciare Precision e Recall.

**Consegna:**
1. Usa il dataset Credit Card Fraud dalla Demo 4
2. Ottieni gli **anomaly scores** da Isolation Forest
3. Testa **diverse soglie** (percentili 90, 95, 97, 99)
4. Trova la soglia che massimizza **F1-Score**
5. Visualizza la curva Precision-Recall

In [ ]:
# ============================================
# ESERCIZIO 26.3 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 26.3 — Threshold Optimization")
print("="*70)

# ============================================
# PASSO 1: Ottieni anomaly scores
# ============================================
# Riusiamo il dataset fraud e il modello già trainato
scores_opt = -iso_fraud.decision_function(X_fraud_scaled)  # Più alto = più anomalo

print(f"\n📊 Distribuzione Anomaly Scores:")
print(f"   - Min: {scores_opt.min():.3f}")
print(f"   - Max: {scores_opt.max():.3f}")
print(f"   - Mean: {scores_opt.mean():.3f}")

# ============================================
# PASSO 2: Test diverse soglie
# ============================================
percentiles = [90, 92, 94, 95, 96, 97, 98, 99]
results_thresh = []

print("\n" + "="*50)
print("Test diverse soglie (percentili)")
print("="*50)

for p in percentiles:
    threshold = np.percentile(scores_opt, p)
    y_pred_thresh = (scores_opt >= threshold).astype(int)
    
    metrics = calc_metrics(y_fraud, y_pred_thresh)
    
    results_thresh.append({
        'percentile': p,
        'threshold': threshold,
        'n_detected': y_pred_thresh.sum(),
        'precision': metrics['Precision'],
        'recall': metrics['Recall'],
        'f1': metrics['F1']
    })
    
    print(f"   P{p}: thresh={threshold:.3f}, detected={y_pred_thresh.sum():3d}, "
          f"P={metrics['Precision']:.2f}, R={metrics['Recall']:.2f}, F1={metrics['F1']:.2f}")

df_thresh = pd.DataFrame(results_thresh)

# ============================================
# PASSO 3: Trova la soglia ottimale
# ============================================
best_idx = df_thresh['f1'].idxmax()
best_percentile = df_thresh.loc[best_idx, 'percentile']
best_threshold = df_thresh.loc[best_idx, 'threshold']
best_f1_opt = df_thresh.loc[best_idx, 'f1']

print(f"\n🥇 SOGLIA OTTIMALE:")
print(f"   - Percentile: {best_percentile}")
print(f"   - Threshold: {best_threshold:.3f}")
print(f"   - F1-Score: {best_f1_opt:.3f}")

# ============================================
# PASSO 4: Precision-Recall Curve
# ============================================
# Calcola curva P-R
precision_curve, recall_curve, thresholds_curve = precision_recall_curve(y_fraud, scores_opt)

# Calcola F1 per ogni punto
f1_curve = 2 * precision_curve * recall_curve / (precision_curve + recall_curve + 1e-10)

# ============================================
# PASSO 5: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Metriche vs Percentile
axes[0].plot(df_thresh['percentile'], df_thresh['precision'], 'bo-', label='Precision', linewidth=2)
axes[0].plot(df_thresh['percentile'], df_thresh['recall'], 'go-', label='Recall', linewidth=2)
axes[0].plot(df_thresh['percentile'], df_thresh['f1'], 'ro-', label='F1-Score', linewidth=2)
axes[0].axvline(x=best_percentile, color='gray', linestyle='--', alpha=0.7)
axes[0].set_xlabel('Percentile Threshold')
axes[0].set_ylabel('Score')
axes[0].set_title('Metriche vs Soglia')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Precision-Recall Curve
axes[1].plot(recall_curve, precision_curve, 'b-', linewidth=2)
axes[1].fill_between(recall_curve, precision_curve, alpha=0.3)
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].grid(True, alpha=0.3)

# Plot 3: Distribuzione scores con soglia ottimale
axes[2].hist(scores_opt[y_fraud==0], bins=50, alpha=0.7, label='Normali', color='blue', density=True)
axes[2].hist(scores_opt[y_fraud==1], bins=50, alpha=0.7, label='Frodi', color='red', density=True)
axes[2].axvline(x=best_threshold, color='green', linestyle='--', linewidth=2, label=f'Soglia ottimale')
axes[2].set_xlabel('Anomaly Score')
axes[2].set_ylabel('Densità')
axes[2].set_title('Distribuzione con Soglia Ottimale')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"""
📌 INTERPRETAZIONE:
   - Soglia bassa (P90): alta Recall, bassa Precision (molti falsi allarmi)
   - Soglia alta (P99): bassa Recall, alta Precision (perse molte frodi)
   - Soglia ottimale (P{best_percentile}): bilanciamento F1
   
💡 NEL BUSINESS:
   - Se i falsi allarmi costano poco → soglia bassa (alta Recall)
   - Se investigare costa molto → soglia alta (alta Precision)
""")

---

# 🎯 5. Conclusione — Cosa Portarsi a Casa

## ✅ Concetti Chiave Appresi

| Concetto | Descrizione |
|----------|-------------|
| **Anomaly Detection** | Trovare punti che si discostano dal comportamento normale |
| **Isolation Forest** | Isola anomalie con pochi split (veloce, robusto) |
| **LOF** | Confronta densità locale vs vicini (buono per anomalie locali) |
| **Contamination** | % attesa di anomalie, parametro critico |

## 🚨 Errori Comuni da Evitare

1. **Ignorare lo sbilanciamento** → Non usare Accuracy, usa F1/PR-AUC
2. **Contamination sbagliato** → Stima accurata della % anomalie
3. **Non validare** → Sempre visualizzare e calcolare metriche
4. **Un solo algoritmo** → Ensemble spesso migliora

## 💡 Quando Usare Quale Algoritmo

```
ISOLATION FOREST:
✅ Default per la maggior parte dei casi
✅ Dataset grandi (scalabile)
✅ Anomalie globali
✅ Quando la velocità conta

LOF:
✅ Anomalie locali/contestuali
✅ Cluster con densità diverse
✅ Dataset medio-piccoli
✅ Quando serve interpretabilità (LOF score = densità)

ENSEMBLE:
✅ Quando vuoi massima copertura
✅ Budget computazionale permette
✅ Anomalie di tipi diversi
```

## 🔗 Ponte verso la Prossima Lezione

Nella **Lezione 27** esploreremo **Unsupervised Feature Engineering**:
- Come creare nuove feature usando tecniche unsupervised
- Embedding e rappresentazioni latenti
- Preparare i dati per modelli supervised

---

# 📚 6. Bignami — Scheda di Riferimento Rapido

## 📖 Definizioni Essenziali

| Termine | Definizione |
|---------|-------------|
| **Anomalia** | Osservazione che si discosta dal comportamento normale |
| **Contamination** | Percentuale attesa di anomalie nel dataset |
| **Anomaly Score** | Valore numerico che indica quanto un punto è anomalo |
| **Point Anomaly** | Singolo punto anomalo |
| **Contextual Anomaly** | Anomalo solo in un contesto specifico |

## 📐 Algoritmi a Confronto

| Algoritmo | Complessità | Anomalie Locali | Scalabilità |
|-----------|-------------|-----------------|-------------|
| Isolation Forest | O(n log n) | Media | Ottima |
| LOF | O(n²) | Eccellente | Media |
| DBSCAN | O(n log n) | Buona | Buona |

## 🔧 Template di Codice

### Isolation Forest
```python
from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 5% anomalie attese
    random_state=42
)

# -1 = anomalia, 1 = normale
y_pred = iso.fit_predict(X)

# Scores (più basso = più anomalo)
scores = iso.decision_function(X)
```

### Local Outlier Factor
```python
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.05
)

y_pred = lof.fit_predict(X)

# LOF scores (più alto = più anomalo)
lof_scores = -lof.negative_outlier_factor_
```

### Valutazione
```python
from sklearn.metrics import precision_recall_curve, f1_score

# Converti a binary (anomalia = 1)
y_pred_binary = (y_pred == -1).astype(int)

# Metriche
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
```

## ✅ Checklist Anomaly Detection

- [ ] Definito cosa significa "anomalia" nel contesto?
- [ ] Stimata la % di anomalie (contamination)?
- [ ] Dati scalati (per LOF)?
- [ ] Algoritmo appropriato scelto?
- [ ] Soglia ottimizzata?
- [ ] Metriche calcolate (P, R, F1)?
- [ ] Risultati visualizzati e validati?

---

**🎉 Fine Lezione 26 — Anomaly Detection Masterclass!**